In [3]:
import pandas as pd
# Defining sets and parameters

# factories
factories = ["IJmuiden","Segal","South Wales"]

# types of rebars and lengths
rebars = {      
    "A": {"length": 2.4},
    "B": {"length": 3.6},
    "C": {"length": 4.2}
} 

# types of long bars and lengths
longbars = {
    1: {"length": 9},
    2: {"length": 12}
}

diameter=0.057 #in meters
density=7.85 #tons/m^3

# production capacity in tonnes per period
production_capacity = {
    "IJmuiden": 12,   
    "Segal": 10,
    "South Wales": 28
}

# customer areas
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]


# time periods
periods = [1,2,3,4]


# demand for each type of rebar
demand = {}
demand["A"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [2, 6, 5, 3],
    "Boenen": [4, 8, 5, 10],
    "Dortmund": [2, 7, 6, 5],
    "Gelsenkirchen": [5, 5, 5, 5],
    "Hagen": [19, 23, 25, 16],
    "Iserlohn": [13, 19, 17, 14],
    "Neuss": [20, 16, 14, 26],
    "Schwerte": [4, 5, 3, 4]
}

demand["B"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [4, 5, 7, 8],
    "Boenen": [5, 8, 12, 13],
    "Dortmund": [4, 5, 8, 10],
    "Gelsenkirchen": [9, 10, 6, 6],
    "Hagen": [15, 33, 31, 33],
    "Iserlohn": [22, 26, 20, 27],
    "Neuss": [12, 23, 30, 30],
    "Schwerte": [2, 8, 2, 6]
}

demand["C"] = {
    "Period": [1, 2, 3, 4],
    "Bochum": [6, 7, 7, 7],
    "Boenen": [6, 10, 15, 12],
    "Dortmund": [7, 6, 4, 12],
    "Gelsenkirchen": [10, 9, 9, 10],
    "Hagen": [12, 35, 33, 38],
    "Iserlohn": [14, 25, 23, 24],
    "Neuss": [22, 32, 31, 31],
    "Schwerte": [5, 6, 7, 2]
}

#demand["B"]["Neuss"][0]

fixed_cost = {
    "IJmuiden": 130,   
    "Segal": 150,
    "South Wales": 100
}

variable_cost = 0.5

distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Now you can access distances like this:
print(distance["Bochum"]["Segal"])  # Output: 203

# possible number of long bars
n = 95
possible_longbars = set(range(1, n + 1))


203


In [40]:
## Import pyomo environment and pyomo optimizer
import pyomo.environ as pe
import pyomo.opt as po
import math

time_period = 1

m = pe.ConcreteModel()
m.x = pe.Var(rebars, longbars, possible_longbars, factories, domain = pe.NonNegativeIntegers)
m.y = pe.Var(longbars, possible_longbars, factories, domain = pe.Binary)
m.z = pe.Var(factories, customers, domain = pe.Binary)

obj_expr = sum(m.z[f,c] * (fixed_cost[f] + variable_cost * distance[c][f] * sum(rebars[r]["length"] * m.x[(r,l,n,f)] for n in possible_longbars for l in longbars for r in rebars)* density * (diameter/2) ** 2 * math.pi) for c in customers for f in factories)
m.obj = pe.Objective(sense = pe.minimize, expr= obj_expr)

# Defining the cuts in long bars
@m.Constraint(longbars, possible_longbars, factories)
def cuts(m, l, n, f):
    return sum(rebars[r]["length"] * m.x[(r,l,n,f)] for r in rebars) <= longbars[l]["length"]*m.y[(l,n,f)]

# Supply constraint
@m.Constraint(factories)
def supply(m, f):
    return density * sum(m.y[(l,n,f)]*longbars[l]["length"] * (diameter/2) ** 2 * math.pi for n in possible_longbars for l in longbars)  <= production_capacity[f]

# Demand constraint
@m.Constraint(factories, rebars)
def demand_constraint(m, f, r):
    return sum(m.x[(r,l,n,f)] for n in possible_longbars for l in longbars) >= sum(demand[r][c][time_period-1] * m.z[(f,c)] for c in customers) 


# Each customer is served by 1 factory only
@m.Constraint(customers)
def one_supplier(m, c):
    return sum(m.z[(f,c)] for f in factories) == 1

solver = po.SolverFactory('gurobi')
result = solver.solve(m, tee = True, options={'TimeLimit':3600})

print(result.solver.status)
print(result.solver.termination_condition)

Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmpsfmla2jo.pyomo.lp
Reading time = 0.01 seconds
x1: 590 rows, 2304 columns, 4656 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 590 rows, 2304 columns and 4656 nonzeros
Model fingerprint: 0x9f538c9c
Model has 13680 quadratic objective terms
Variable types: 0 continuous, 2304 integer (594 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+01]
  Objective range  [1e+02, 2e+02]
  QObjective range [7e+00, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve time: 0.01s
Presolved: 614 rows, 2328 columns, 18384 nonzeros
Variable types: 24 continuous, 2304 integer (594 binary)
Found heuristic soluti

In [42]:
#debug
#m.pprint()
# Print only nonzero variables
print("\nNonzero Decision Variables:")

for v in m.component_objects(pe.Var, active=True):
    print(f"\nVariable: {v.name}")
    for index in v:
        if abs(v[index].value) > 1e-6:  # Ignore near-zero values
            print(f"{index} = {v[index].value}")




Nonzero Decision Variables:

Variable: x
('C', 1, 1, 'Segal') = 1.0
('C', 1, 2, 'Segal') = 1.0
('C', 1, 11, 'Segal') = 1.0
('C', 1, 16, 'Segal') = 2.0
('C', 1, 20, 'Segal') = 2.0
('C', 1, 29, 'South Wales') = 2.0
('C', 1, 35, 'South Wales') = 2.0
('C', 1, 42, 'Segal') = 2.0
('C', 1, 45, 'Segal') = 2.0
('C', 1, 49, 'Segal') = 2.0
('C', 1, 54, 'South Wales') = 2.0
('C', 1, 71, 'Segal') = 2.0
('C', 1, 75, 'Segal') = 2.0
('C', 1, 79, 'Segal') = 2.0
('C', 1, 81, 'Segal') = 2.0
('C', 1, 84, 'Segal') = 2.0
('C', 1, 85, 'Segal') = 2.0
('C', 1, 86, 'Segal') = 2.0
('C', 1, 87, 'Segal') = 2.0
('C', 1, 88, 'Segal') = 2.0
('C', 1, 89, 'Segal') = 2.0
('C', 1, 90, 'Segal') = 2.0
('C', 1, 91, 'Segal') = 2.0
('C', 1, 92, 'Segal') = 2.0
('C', 1, 93, 'Segal') = 2.0
('C', 1, 94, 'Segal') = 1.0
('C', 1, 95, 'Segal') = 1.0
('C', 2, 4, 'IJmuiden') = 2.0
('C', 2, 9, 'Segal') = 1.0
('C', 2, 30, 'IJmuiden') = 2.0
('C', 2, 57, 'IJmuiden') = 2.0
('C', 2, 69, 'IJmuiden') = 2.0
('C', 2, 74, 'IJmuiden') = 2.0
('C',

In [58]:
'''
tot =0
for f in factories:
    for l in longbars:
        for n in possible_longbars:
            
                tot = tot + m.x[('C',l,n,f)].value

print(tot)
'''

"\ntot =0\nfor f in factories:\n    for l in longbars:\n        for n in possible_longbars:\n            \n                tot = tot + m.x[('C',l,n,f)].value\n\nprint(tot)\n"

In [38]:
m.pprint()

3 Var Declarations
    x : Size=1710, Index={C, A, B}*{1, 2}*{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95}*{IJmuiden, Segal, South Wales}
        Key                         : Lower : Value : Upper : Fixed : Stale : Domain
            ('A', 1, 1, 'IJmuiden') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
               ('A', 1, 1, 'Segal') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
         ('A', 1, 1, 'South Wales') :     0 :   3.0 :  None : False : False : NonNegativeIntegers
            ('A', 1, 2, 'IJmuiden') :     0 :   1.0 :  None : False : False : NonNegativeIntegers
               ('A', 1, 2, 'Segal') :     0 :   0.0 :  None